    Τμήμα Πληροφορικής και Τηλεπικοινωνιών - Άρτα 
    Πανεπιστήμιο Ιωαννίνων 

    Γκόγκος Χρήστος 
    http://chgogos.github.io/
    Εαρινό εξάμηνο 2020-2021

# Νήματα (threads)

Νήμα είναι μια διακριτή ροή εκτέλεσης ενός προγράμματος. Ένα πρόγραμμα μπορεί να εκτελεί ταυτόχρονα (αλλά όχι την ίδια χρονική στιγμή στην Python) περισσότερα από ένα νήματα.

H Python υποστηρίζει τον ταυτοχρονισμό με τη χρήση νημάτων και την παραλληλία με τη χρήση πολυδιεργασίας (multiprocessing)

    I/O bound (εργασίες που συχνά περιμένουν την εκτέλεση εξωτερικών γεγονότων) --> threading 
    CPU bound (εργασίες που απασχολούν σημαντικά την CPU) --> multiprocessing

In [ ]:
# παράδειγμα 1
# δημιουργία και εκτέλεση ενός νήματος

import threading

def work(tid):
    internal_thread_id = threading.get_ident()
    print(f"Μήνυμα από το νήμα {tid} - {internal_thread_id}")

t = threading.Thread(target=work, args=(1,))
t.start()

t.join()
print(f"Μήνυμα από το κύριο νήμα {threading.get_ident()}")

In [ ]:
# παράδειγμα 2
# βήμα 1/5 
# εκτέλεση 2 διαδοχικές φορές ενός κώδικα με χρόνο εκτέλεσης 1 δευτερόλεπτο.

import time

start = time.perf_counter()

def func():
    print('Ύπνος')
    time.sleep(1) # προσομειώνει μια I/O bound εργασία
    print('Αφύπνιση')

func()
func()
finish = time.perf_counter()
print(f'Τερματισμός μετά από {round(finish-start, 2)} δευτερόλεπτα')

In [ ]:
# βήμα 2/5
# εκτέλεση 2 φορές ενός κώδικα με χρόνο εκτέλεσης 1 δευτερόλεπτο, κάθε εκτέλεση πραγματοποιείται σε διαφορετικό thread

import threading
import time


start = time.perf_counter()

def func():
    print('Ύπνος')
    time.sleep(1)
    print('Αφύπνιση')

t1 = threading.Thread(target=func)
t2 = threading.Thread(target=func)
t1.start()
t2.start()

t1.join()
t2.join()

finish = time.perf_counter()
print(f'Τερματισμός μετά από {round(finish-start, 2)} δευτερόλεπτα')

In [ ]:
# βήμα 3/5
# Εκτέλεση 10 νημάτων που τοποθετούνται σε λίστα, σε κάθε νήμα αποδίδεται ένας αριθμός νήματος

import threading
import time

start = time.perf_counter()

def func(tid, seconds):
    print(f'Το νήμα {tid} θα κοιμηθεί για {seconds} δευτερόλεπτα')
    time.sleep(seconds)
    print(f'Αφύπνιση νήματος {tid}')

threads = []
for i in range(10):
    t = threading.Thread(target=func, args=[i, 1])
    t.start()
    threads.append(t)

for t in threads:
    t.join()

finish = time.perf_counter()
print(f'Τερματισμός μετά από {round(finish-start, 2)} δευτερόλεπτα')

In [ ]:
# βήμα 4/5
# Εκτέλεση 10 νημάτων, το κάθε νήμα επιστρέφει ως αποτέλεσμα τον αριθμό νήματος του
# ThreadPoolExecutor (χρήση με context manager) 
# >= Python 3.2

import concurrent.futures
import time

start = time.perf_counter()

def func(tid, seconds):
    print(f'Το νήμα {tid} θα κοιμηθεί για {seconds} δευτερόλεπτα')
    time.sleep(seconds)
    print(f'Αφύπνιση νήματος {tid}')
    return tid

with concurrent.futures.ThreadPoolExecutor() as executor:
    secs = [5,4,3,2,1]
    results = [executor.submit(func, tid=i, seconds=sec) for i, sec in zip(range(5), secs)]
    
    for f in concurrent.futures.as_completed(results):
        print(f'Αποτέλεσμα={f.result()}')

finish = time.perf_counter()
print(f'Τερματισμός μετά από {round(finish-start, 2)} δευτερόλεπτα')

In [ ]:
# βήμα 5/5
# Εκτέλεση 10 νημάτων, το κάθε νήμα επιστρέφει ως αποτέλεσμα τον αριθμό νήματος του
# Απούστερος τρόπος με χρήση executor.map

import concurrent.futures
import time

start = time.perf_counter()

def func(tid, seconds):
    print(f'Το νήμα {tid} θα κοιμηθεί για {seconds} δευτερόλεπτα')
    time.sleep(seconds)
    print(f'Αφύπνιση νήματος {tid}')
    return tid

with concurrent.futures.ThreadPoolExecutor() as executor:
    secs = [5,4,3,2,1]
    results = executor.map(func, range(5), secs)
    
    for result in results:     
        print(f'Αποτέλεσμα={result}')

finish = time.perf_counter()
print(f'Τερματισμός μετά από {round(finish-start, 2)} δευτερόλεπτα')

In [ ]:
# Παράδειγμα 3
# Δύο νήματα, το πρώτο αυξάνει την global μεταβλητή counter κατά 1, 1000000 φορές και το δεύτερο νήμα μειώνει τη μεταβλητή counter κατά 1, 1000000 φορές. Συνεπώς, η τελική αναμενόμενη τιμή του counter είναι 0.

import threading
import time

def work1(lock):
    global counter 
    for _ in range(1000000):
        with lock:
            counter +=1

def work2(lock):
    global counter
    for _ in range(1000000):
        with lock:
            counter -=1

start = time.perf_counter()

counter = 0
lock = threading.Lock()

t1 =  threading.Thread(target=work1, args=[lock])
t2 =  threading.Thread(target=work2, args=[lock])
t1.start()
t2.start()

t1.join()
t2.join()

finish = time.perf_counter()
print(f'Τερματισμός μετά από {round(finish-start, 2)} δευτερόλεπτα')
print(f'Counter={counter}')

## References

* [Real Python ](https://realpython.com/intro-to-python-threading/)
* [Python Land - Concurrency in Python](https://medium.com/pythonland/concurrency-in-python-832dc54d35f6)